In [1]:
import pandas as pd
import numpy as np

In [3]:
test = pd.read_csv('csv_files/test_final_100k.csv')

A continuación vamos a realizar las mismas operaciones con los dataframes de postulantes y avisos que realizamos en el notebook principal, a fin de tener un dataframe test con los features correspondientes para que los algoritmos de Machine Learning puedan predecir los resultados.

No realizamos comentarios respecto a las operaciones ya que estos se encuentran en el notebook principal.

## Información de los postulantes

### DF 1: Educación de los postulantes

In [5]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')

postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)

postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Género y Edad de los postulantes

In [8]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')

postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)
postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)

postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'NO_DECLARA'), 'sexo'] = 0
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'FEM'),'sexo'] = 1
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'MASC'),'sexo'] = 2

postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null int64
rango_edad      478001 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


### Merge de los DF 1 y 2

In [14]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='outer')

postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487459 entries, 0 to 487458
Data columns (total 5 columns):
idpostulante       487459 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
sexo               478001 non-null float64
rango_edad         478001 non-null float64
dtypes: float64(4), object(1)
memory usage: 22.3+ MB


## Detalles de los avisos

In [11]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles = avisos_detalles.drop_duplicates('idaviso')

avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle', 'descripcion', 'denominacion_empresa'}, axis=1, inplace=True)

avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 6 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
dtypes: int64(4), object(2)
memory usage: 1.3+ MB


## Merge del Test Final con la información de los postulantes

In [15]:
test_final = pd.merge(test, postulantes, on='idpostulante', how='left')
test_final.sample(5)

,id,idaviso,idpostulante,estudios,esta_estudiando,sexo,rango_edad
20294,20294,1112253226,81Bljx,2.0,0.0,2.0,3.0
94682,94682,1112485262,Y90Neq,6.0,0.0,2.0,3.0
60110,60110,1112454794,DrzdJJd,4.0,1.0,1.0,3.0
18025,18025,1112208702,EeOVMJ,3.0,0.0,1.0,4.0
49603,49603,1112438273,Z9rG9z,4.0,0.0,2.0,4.0


In [16]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           94388 non-null float64
esta_estudiando    94388 non-null float64
sexo               97019 non-null float64
rango_edad         97019 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 6.1+ MB


In [17]:
test_final.loc[(test_final['estudios'].isnull()), 'estudios'] = 0
test_final.loc[(test_final['esta_estudiando'].isnull()), 'esta_estudiando'] = 0
test_final.loc[(test_final['sexo'].isnull()), 'sexo'] = 0
test_final.loc[(test_final['rango_edad'].isnull()), 'rango_edad'] = 0

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null float64
rango_edad         100000 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 6.1+ MB


## Merge del Test Final con los detalles de los avisos

In [18]:
test_final = pd.merge(test_final, avisos_detalles, on='idaviso', how='left')
test_final.sample(5)

,id,idaviso,idpostulante,estudios,esta_estudiando,sexo,rango_edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
97873,97873,1112503097,19GdGL,3.0,0.0,1.0,4.0,Auxiliares de instalaciones Pilar y Moreno,3.0,9.0,2.0,Telecomunicaciones
5058,5058,1111513745,1jxR8Q,0.0,0.0,2.0,3.0,Secretaria Administrativa / Recepcionista  Ro...,3.0,10.0,3.0,Recepcionista
47462,47462,1112431336,NPx4wJ,3.0,0.0,1.0,3.0,Analista Comercio Exterior / Internacional,3.0,10.0,3.0,Compras Internacionales/Importación
63107,63107,1112457238,N5EvOV,3.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN
79351,79351,1112464687,RzMKKwq,4.0,1.0,2.0,2.0,Ejecutivo Comercial,3.0,10.0,2.0,Comercial


In [19]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null float64
rango_edad         100000 non-null float64
titulo             95552 non-null object
nombre_zona        95552 non-null float64
tipo_de_trabajo    95552 non-null float64
nivel_laboral      95552 non-null float64
nombre_area        95552 non-null object
dtypes: float64(7), int64(2), object(3)
memory usage: 9.9+ MB


In [20]:
test_final.loc[(test_final['nombre_zona'].isnull()), 'nombre_zona'] = 0
test_final.loc[(test_final['tipo_de_trabajo'].isnull()), 'tipo_de_trabajo'] = 0
test_final.loc[(test_final['nivel_laboral'].isnull()), 'nivel_laboral'] = 0

test_final.rename(columns={'estudios':'nivel_estudios'}, inplace=True)

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
nivel_estudios     100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null float64
rango_edad         100000 non-null float64
titulo             95552 non-null object
nombre_zona        100000 non-null float64
tipo_de_trabajo    100000 non-null float64
nivel_laboral      100000 non-null float64
nombre_area        95552 non-null object
dtypes: float64(7), int64(2), object(3)
memory usage: 9.9+ MB


In [21]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
nivel_estudios     100000 non-null category
esta_estudiando    100000 non-null category
sexo               100000 non-null category
rango_edad         100000 non-null category
titulo             95552 non-null object
nombre_zona        100000 non-null category
tipo_de_trabajo    100000 non-null category
nivel_laboral      100000 non-null category
nombre_area        95552 non-null object
dtypes: category(7), int64(2), object(3)
memory usage: 5.2+ MB


In [22]:
test_final = test_final[['id','idaviso','idpostulante','rango_edad','sexo',\
                         'nivel_estudios','esta_estudiando','titulo','tipo_de_trabajo',\
                         'nivel_laboral','nombre_zona','nombre_area']]

## Exportamos a un CSV File el test final

In [23]:
test_final.to_csv('csv_files/test_final.csv', encoding='utf-8', index=False)

## Aplicamos Feature Engineering

A continuación, añadiremos los features que desarrollamos en el notebook 'feature_engineering' al archivo test final. No emitiremos comentarios sobre las operaciones realizadas, ya que éstos se encuentran en el notebook mencionado.

In [37]:
test_final = pd.read_csv('csv_files/test_final.csv')

In [39]:
estudios_vs_laboral_t = pd.read_csv('csv_files/estudios_vs_laboral_t.csv')

estudiando_vs_tipo_trabajo_t = pd.read_csv('csv_files/estudiando_vs_tipo_trabajo_t.csv')

edad_vs_tipo_trabajo_t = pd.read_csv('csv_files/edad_vs_tipo_trabajo_t.csv')

#edad_vs_zona_t = pd.read_csv('csv_files/edad_vs_zona_t.csv')

In [40]:
n_estudios = 7
for index, row in estudios_vs_laboral_t.iterrows():
    for i in range(1,n_estudios+1):
        peso = row[i]
        nivel_estudios = i
        nivel_laboral = index+1
        relacion = (test_final['nivel_estudios'] == nivel_estudios) & (test_final['nivel_laboral'] == nivel_laboral)
        test_final.loc[(relacion), 'estudios_vs_laboral'] = peso

In [41]:
n_trabajos = 10
for index, row in estudiando_vs_tipo_trabajo_t.iterrows():
    for i in range(1,n_trabajos+1):
        peso = row[i]
        tipo_trabajo = i
        esta_estudiando = index
        relacion = (test_final['tipo_de_trabajo'] == tipo_trabajo) & (test_final['esta_estudiando'] == esta_estudiando)
        test_final.loc[(relacion), 'estudia_vs_tipo_trabajo'] = peso

In [42]:
n_trabajos = 10
for index, row in edad_vs_tipo_trabajo_t.iterrows():
    for i in range(1,n_trabajos+1):
        peso = row[i]
        tipo_trabajo = i
        rango_edad = index+1
        relacion = (test_final['tipo_de_trabajo'] == tipo_trabajo) & (test_final['rango_edad'] == rango_edad)
        test_final.loc[(relacion), 'edad_vs_tipo_trabajo'] = peso

In [43]:
#n_zonas = 3
#for index, row in edad_vs_zona_t.iterrows():
#    for i in range(2,n_zonas+1):
#        peso = row[i]
#        nombre_zona = i
#        rango_edad = index+1
#        relacion = (test_final['nombre_zona'] == nombre_zona) & (test_final['rango_edad'] == rango_edad)
#        test_final.loc[(relacion), 'edad_vs_zona'] = peso

In [44]:
test_final.loc[(test_final['estudios_vs_laboral'].isnull()), 'estudios_vs_laboral'] = 0
test_final.loc[(test_final['estudia_vs_tipo_trabajo'].isnull()), 'estudia_vs_tipo_trabajo'] = 0
test_final.loc[(test_final['edad_vs_tipo_trabajo'].isnull()), 'edad_vs_tipo_trabajo'] = 0
#test_final.loc[(test_final['edad_vs_zona'].isnull()), 'edad_vs_zona'] = 0

In [45]:
test_final.sample(5)

,id,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,estudios_vs_laboral,estudia_vs_tipo_trabajo,edad_vs_tipo_trabajo
64097,64097,1112457811,VNPkxQM,1.0,1.0,4.0,1.0,GESTORES DE COBRANZA TELEFONICOS,9.0,1.0,3.0,Call Center,0.068507,0.466237,0.502475
71003,71003,1112461994,vVYom3e,2.0,2.0,4.0,1.0,OPERADORES TELEFÓNICOS FULLFILMENT,9.0,3.0,3.0,Atención al Cliente,0.640946,0.466237,0.258020
43882,43882,1112425633,VNPR04j,3.0,1.0,6.0,0.0,NaN,0.0,0.0,0.0,NaN,0.000000,0.000000,0.000000
63189,63189,1112457348,vVjGVeo,1.0,2.0,4.0,1.0,Personal de carga y descarga.,10.0,3.0,3.0,Mantenimiento y Limpieza,0.640946,0.347169,0.315631
16118,16118,1112159641,1e1a2N,4.0,1.0,3.0,0.0,1129 Operario de Producción para Textil de Tigre.,10.0,3.0,3.0,Producción,0.659570,0.652831,0.101802


In [46]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
id                         100000 non-null int64
idaviso                    100000 non-null int64
idpostulante               100000 non-null object
rango_edad                 100000 non-null float64
sexo                       100000 non-null float64
nivel_estudios             100000 non-null float64
esta_estudiando            100000 non-null float64
titulo                     95552 non-null object
tipo_de_trabajo            100000 non-null float64
nivel_laboral              100000 non-null float64
nombre_zona                100000 non-null float64
nombre_area                95552 non-null object
estudios_vs_laboral        100000 non-null float64
estudia_vs_tipo_trabajo    100000 non-null float64
edad_vs_tipo_trabajo       100000 non-null float64
dtypes: float64(10), int64(2), object(3)
memory usage: 11.4+ MB


## Exportamos a un CSV File el test final con los nuevos features

In [47]:
test_final.to_csv('csv_files/test_final_featured.csv', encoding='utf-8', index=False)